In [13]:
import pandas as pd
from urllib.parse import urlparse, parse_qs
from googleapiclient.discovery import build
from tqdm.notebook import tqdm
import math

# 데이터 로드
df = pd.read_csv("days_before_launch_data.csv")

In [16]:
df['video_link']

0       igfmUe-IBeQ
1       8anIGeV18HE
2       afZAmzYWEu4
3       2f8S7gPK4b4
4       qDCADmoCec8
           ...     
5325    u8lYaO-W7gc
5326    EDseppt_CGM
5327    xjVkGQipNk0
5328    uy8I8BemMtY
5329    9QeNEoEWxqg
Name: video_link, Length: 5330, dtype: object

In [14]:
df['video_id'] = df['video_link']

In [17]:
df.head()

publisher                 game_names      video_type   video_link  \
0    Capcom              Lost Planet 3         Trailer  igfmUe-IBeQ   
1    Capcom              Lost Planet 3  Gameplay video  8anIGeV18HE   
2    Capcom              Lost Planet 3  Gameplay video  afZAmzYWEu4   
3    Capcom  Call of Duty: Finest Hour  Gameplay Video  2f8S7gPK4b4   
4    Capcom  Call of Duty: Finest Hour         Trailer  qDCADmoCec8   

                                     video_url game_release_date  \
0  https://www.youtube.com/watch?v=igfmUe-IBeQ        2013-08-26   
1  https://www.youtube.com/watch?v=8anIGeV18HE        2013-08-26   
2  https://www.youtube.com/watch?v=afZAmzYWEu4        2013-08-26   
3  https://www.youtube.com/watch?v=2f8S7gPK4b4        2004-11-16   
4  https://www.youtube.com/watch?v=qDCADmoCec8        2004-11-16   

  youtube_uploaded_date  days_before_launch     video_id  
0   2012-08-14 11:14:55              -377.0  igfmUe-IBeQ  
1   2016-02-28 19:27:03               916.0  8anIGeV18HE  
2   2016-02-29 13:49:12               917.0  afZAmzYWEu4  
3   2019-12-16 17:30:00              5508.0  2f8S7gPK4b4  
4   2011-05-19 22:57:07              2375.0  qDCADmoCec8

In [ ]:
# API 키 세팅
api_key = "API_KEY"
youtube = build("youtube", "v3", developerKey=api_key)

In [19]:
# 배치 크기 및 ID 목록 준비
batch_size = 50
video_ids = df["video_id"].dropna().unique().tolist()

# 결과 담을 dict
stats_dict = {}

for i in tqdm(range(0, len(video_ids), batch_size)):
    batch = video_ids[i : i + batch_size]
    res = youtube.videos().list(
        part="statistics,snippet",
        id=",".join(batch)
    ).execute()

    for item in res.get("items", []):
        vid = item["id"]
        stat = item["statistics"]
        chan = item["snippet"]["channelId"]
        stats_dict[vid] = {
            "viewCount":    int(stat.get("viewCount", 0)),
            "likeCount":    int(stat.get("likeCount", 0)),
            "commentCount": int(stat.get("commentCount", 0)),
            "channelId":    chan
        }


  0%|          | 0/72 [00:00<?, ?it/s]

In [20]:
# 채널 ID 추출 & 중복 제거
channel_ids = list({v["channelId"] for v in stats_dict.values()})

subs_dict = {}
for i in tqdm(range(0, len(channel_ids), batch_size)):
    batch = channel_ids[i : i + batch_size]
    res = youtube.channels().list(
        part="statistics",
        id=",".join(batch)
    ).execute()

    for item in res.get("items", []):
        cid = item["id"]
        subs_dict[cid] = int(item["statistics"].get("subscriberCount", 0))


  0%|          | 0/22 [00:00<?, ?it/s]

In [21]:
# 빈 컬럼 생성
df["viewCount"]       = 0
df["likeCount"]       = 0
df["commentCount"]    = 0
df["subscriberCount"] = 0

# 채워넣기
for idx, vid in enumerate(df["video_id"]):
    if vid in stats_dict:
        df.at[idx, "viewCount"]     = stats_dict[vid]["viewCount"]
        df.at[idx, "likeCount"]     = stats_dict[vid]["likeCount"]
        df.at[idx, "commentCount"]  = stats_dict[vid]["commentCount"]
        df.at[idx, "subscriberCount"] = subs_dict.get(stats_dict[vid]["channelId"], 0)




In [22]:
df.head(20)

publisher                                game_names            video_type  \
0     Capcom                             Lost Planet 3               Trailer   
1     Capcom                             Lost Planet 3        Gameplay video   
2     Capcom                             Lost Planet 3        Gameplay video   
3     Capcom                 Call of Duty: Finest Hour        Gameplay Video   
4     Capcom                 Call of Duty: Finest Hour               Trailer   
5     Capcom                 Call of Duty: Finest Hour               Trailer   
6     Capcom                 Call of Duty: Finest Hour               Trailer   
7     Capcom                         Red Dead Revolver               Trailer   
8     Capcom                         Red Dead Revolver        Gameplay Video   
9     Capcom                    Street Fighter Alpha 3               Trailer   
10    Capcom                    Street Fighter Alpha 3        Gameplay Video   
11    Capcom                    Street Fighter Alpha 3        Gameplay Video   
12    Capcom  Tatsunoko vs. Capcom: Ultimate All Stars               Trailer   
13    Capcom                 Monster Hunter 3 Ultimate               Trailer   
14    Capcom                 Monster Hunter 3 Ultimate               Trailer   
15    Capcom                 Monster Hunter 3 Ultimate               Trailer   
16    Capcom                        Monster Hunter Tri               Trailer   
17    Capcom                        Monster Hunter Tri               Trailer   
18    Capcom                         Street Fighter IV  Announcement Trailer   
19    Capcom                         Street Fighter IV               Trailer   

     video_link                                    video_url  \
0   igfmUe-IBeQ  https://www.youtube.com/watch?v=igfmUe-IBeQ   
1   8anIGeV18HE  https://www.youtube.com/watch?v=8anIGeV18HE   
2   afZAmzYWEu4  https://www.youtube.com/watch?v=afZAmzYWEu4   
3   2f8S7gPK4b4  https://www.youtube.com/watch?v=2f8S7gPK4b4   
4   qDCADmoCec8  https://www.youtube.com/watch?v=qDCADmoCec8   
5   t7GTci0mT2Q  https://www.youtube.com/watch?v=t7GTci0mT2Q   
6   0JzW38drYic  https://www.youtube.com/watch?v=0JzW38drYic   
7   N-exuahRl2g  https://www.youtube.com/watch?v=N-exuahRl2g   
8   _8yxEqmxOvE  https://www.youtube.com/watch?v=_8yxEqmxOvE   
9   xJwwVeQuZ6I  https://www.youtube.com/watch?v=xJwwVeQuZ6I   
10  mDQSK4aSoWM  https://www.youtube.com/watch?v=mDQSK4aSoWM   
11  hY0lAuhQwzk  https://www.youtube.com/watch?v=hY0lAuhQwzk   
12  qeq-SVAnzNs  https://www.youtube.com/watch?v=qeq-SVAnzNs   
13  z648qIBejT0  https://www.youtube.com/watch?v=z648qIBejT0   
14  7uzafxEHbG0  https://www.youtube.com/watch?v=7uzafxEHbG0   
15  EI75zyCLCG0  https://www.youtube.com/watch?v=EI75zyCLCG0   
16  QE_629ae3iE  https://www.youtube.com/watch?v=QE_629ae3iE   
17  8l3f3csAZn8  https://www.youtube.com/watch?v=8l3f3csAZn8   
18  DJ3ONTRFSqE  https://www.youtube.com/watch?v=DJ3ONTRFSqE   
19  JWhOsX_LAfw  https://www.youtube.com/watch?v=JWhOsX_LAfw   

   game_release_date youtube_uploaded_date  days_before_launch     video_id  \
0         2013-08-26   2012-08-14 11:14:55              -377.0  igfmUe-IBeQ   
1         2013-08-26   2016-02-28 19:27:03               916.0  8anIGeV18HE   
2         2013-08-26   2016-02-29 13:49:12               917.0  afZAmzYWEu4   
3         2004-11-16   2019-12-16 17:30:00              5508.0  2f8S7gPK4b4   
4         2004-11-16   2011-05-19 22:57:07              2375.0  qDCADmoCec8   
5         2004-11-16   2010-10-16 17:51:59              2160.0  t7GTci0mT2Q   
6         2004-11-16   2010-10-13 20:50:34              2157.0  0JzW38drYic   
7         2004-05-02   2007-04-14 23:45:30              1077.0  N-exuahRl2g   
8         2004-05-02   2018-08-07 08:00:02              5210.0  _8yxEqmxOvE   
9         1998-06-29   2010-07-02 18:24:57              4386.0  xJwwVeQuZ6I   
10        1998-06-29   2022-10-29 21:19:44              8888.0  mDQSK4aSoWM   
11        1998-06-29   2018-0

In [23]:
del df['video_id']

In [24]:
# 최종 저장
df.to_csv("AAA_final.csv", index=False)